In [413]:
import random
import heapq
from dataclasses import dataclass, field
from typing import List, Tuple, Callable, Any, Set
import numpy as np
from time import time
from tabulate import tabulate
import threading
import collections
from math import ceil
from enum import Enum
from functools import partial
from copy import deepcopy

In [414]:
class LightsOutPuzzle:
    def __init__(self, board: List[List[int]]):
        self.board = np.array(board, dtype=np.uint8)
        self.size = len(board)

    def toggle(self, x, y):
        for dx, dy in [(0, 0), (1, 0), (-1, 0), (0, 1), (0, -1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                self.board[nx, ny] ^= 1

    def is_solved(self):
        return np.all(self.board == 0)

    def get_moves(self):
        return [(x, y) for x in range(self.size) for y in range(self.size)]

    def __str__(self):
        return '\n'.join(' '.join(str(cell) for cell in row) for row in self.board)

In [415]:
def create_random_board(size: int, seed: int, num_toggles: int = None):
    random.seed(time() if seed is None else seed) 
    
    board = [[0 for _ in range(size)] for _ in range(size)]
    puzzle = LightsOutPuzzle(board)

    if num_toggles is None:
        num_toggles = random.randint(1, size * size)

    for _ in range(num_toggles):
        x = random.randint(0, size - 1)
        y = random.randint(0, size - 1)
        puzzle.toggle(x, y)

    return puzzle.board

In [416]:
def show_solution(
    puzzle: LightsOutPuzzle,
    solution: List[Tuple[int, int]],
    algorithm_name: str,
    nodes_visited: int,
    show_steps: bool = False,
):
    print(f"\nSolving with {algorithm_name}:")
    if solution is None:
        print("No solution found.")
        return
    print(f"Solution: {solution}")
    print(f"Nodes visited: {nodes_visited}")
    if show_steps:
        print("\nSolution steps:")
        for i, move in enumerate(solution):
            print(f"\nStep {i+1}: Toggle position {move}")
            puzzle.toggle(*move)
            print(puzzle)
        print(
            "\nFinal state (solved):"
            if puzzle.is_solved()
            else "\nFinal state (not solved):"
        )
        print(puzzle)

In [417]:
tests = [
    create_random_board(3, 42),
    create_random_board(3, 41),
    create_random_board(3, 42, 5),
    create_random_board(4, 42),
    create_random_board(4, 41),
    create_random_board(4, 42, 5),
    #create_random_board(5, 42),
    #create_random_board(5, 41),
    #create_random_board(5, 42, 5),
]

In [418]:
def astar_solve(puzzle: LightsOutPuzzle, heuristic: Callable[[LightsOutPuzzle], int]):
    nodes_checked = 0 
    
    frontier = []
    heapq.heappush(frontier, (0, tuple(map(tuple, puzzle.board)), []))  # Convert to tuple of tuples

    explored = set()

    while frontier:
        f, current_board, path = heapq.heappop(frontier)
        nodes_checked += 1 

        if all(value == 0 for row in current_board for value in row):  
            return path, nodes_checked 

        if current_board in explored:
            continue

        explored.add(current_board)
        
        for move in puzzle.get_moves():
            
            new_board = np.array(current_board)  
            puzzle_copy = LightsOutPuzzle(new_board)
            puzzle_copy.toggle(move[0], move[1])
            new_path = path + [move]

            g = len(new_path)  
            h = heuristic(puzzle_copy)  
            f = g + h 

            heapq.heappush(frontier, (f, tuple(map(tuple, puzzle_copy.board)), new_path))  # Convert to tuple of tuples

    return None, nodes_checked 

In [419]:
def manhattan_heuristic(puzzle):
    total_distance = 0
    for x in range(puzzle.size):
        for y in range(puzzle.size):
            if puzzle.board[x, y] == 1:
                total_distance += x + y
    return total_distance

def lights_on_heuristic(puzzle):
    return np.sum(puzzle.board) 

def corner_heuristic(puzzle):
    corners = [(0, 0), (0, puzzle.size - 1), (puzzle.size - 1, 0), (puzzle.size - 1, puzzle.size - 1)]
    return sum(puzzle.board[x, y] for x, y in corners) * 2

heuristics = [manhattan_heuristic , lights_on_heuristic]

In [420]:
def weighted_manhattan_heuristic(puzzle , weight):
    return weight*manhattan_heuristic(puzzle)

def weighted_corner_heuristic(puzzle , weight):
    return weight*corner_heuristic(puzzle)

weighted_heuristics = [weighted_manhattan_heuristic , weighted_lights_on_heuristic]
weights = [2 , 8]

In [421]:
class SearchStatus(Enum):
    SOLVED = "Solved"
    TIMEOUT = "Timeout"
    NO_SOLUTION = "No Solution"

@dataclass
class SearchResult:
    result: SearchStatus
    solution: Any = None
    nodes_visited: int = None
    time: float = None

    def __str__(self):
        output = f"Result: {self.result.value}\n"
        if self.result == SearchStatus.SOLVED:
            output += f"Solution: {self.solution}\n"
        if self.result != SearchStatus.TIMEOUT:
            output += f"Nodes Visited: {self.nodes_visited}\n"
            output += f"Time Taken: {self.time:.3f} seconds\n"

        return output


DEFAULT_SEARCH_RESULT = SearchResult(SearchStatus.TIMEOUT, None, None, float("inf"))

In [422]:
import time
import threading
from typing import Callable

def run_searches(search_fn: Callable, search_para, name: str, time_limit: int):
    result = {'name': name, 'solution': None, 'nodes_checked' : 0 , 'time': None , 'steps': 0}

    def run():
        nonlocal result
        start_time = time.time()
        try:
            solution , nodes_checked = search_fn(*search_para)  
            total_time = time.time() - start_time  
            result['solution'] = solution
            result['nodes_checked'] = nodes_checked
            result['time'] = total_time
            result['steps'] = len(solution)
            
            print(f"{name}: {result['time']:.3f} seconds")
            print(f"solution: {result['solution']}")
            print(f"steps: {result['steps']}")
            print(f"nodes_checked: {result['nodes_checked']}\n\n")
    
        except Exception as e:  
            print(f"Error occurred while running {name}: {e}")
            result['solution'] = None
            
          
    search_thread = threading.Thread(target=run)
    search_thread.start()
    
    search_thread.join(timeout=time_limit)

    if search_thread.is_alive():
        print(f"Search exceeded time limit of {time_limit} seconds for {name}!")
        return result
    
    return result

In [423]:
def run_test(puzzle, heuristics : List[Callable[[Any], int]] , time_limit=180):
    
    print(f"Running A* tests on puzzle:\n{puzzle}")
    puzzle = LightsOutPuzzle(puzzle)
    astar_results = []

    for heuristic in heuristics:
        astar_results.append(
            run_searches(astar_solve, (deepcopy(puzzle), heuristic), name=f"A* ({heuristic.__name__})", time_limit=time_limit)
        )

    weighted_astar_results = []
    for heuristic in weighted_heuristics:
        for weight in weights:
            weighted_astar_results.append(
              run_searches(
                  astar_solve,
                  (deepcopy(puzzle), partial(heuristic, weight = weight)),
                  name=f"weighted A*({heuristic.__name__}, weight = {weight})",
                  time_limit=time_limit,
              )
          )
    
    return ( 
        test,
        *astar_results,
        *weighted_astar_results
    )



In [424]:
results = [] 

for test in tests:
    result = results.append(run_test(test, heuristics, 180))
    

Running A* tests on puzzle:
[[1 1 1]
 [1 1 1]
 [1 0 0]]
A* (manhattan_heuristic): 0.001 seconds
solution: [(0, 2), (1, 0)]
steps: 2
nodes_checked: 3


A* (lights_on_heuristic): 0.001 seconds
solution: [(1, 0), (0, 2)]
steps: 2
nodes_checked: 6


weighted A*(weighted_manhattan_heuristic, weight = 2): 0.002 seconds
solution: [(0, 2), (1, 0)]
steps: 2
nodes_checked: 3


weighted A*(weighted_manhattan_heuristic, weight = 8): 0.000 seconds
solution: [(0, 2), (1, 0)]
steps: 2
nodes_checked: 3


weighted A*(weighted_lights_on_heuristic, weight = 2): 0.002 seconds
solution: [(1, 0), (0, 2)]
steps: 2
nodes_checked: 6


weighted A*(weighted_lights_on_heuristic, weight = 8): 0.002 seconds
solution: [(1, 0), (0, 2)]
steps: 2
nodes_checked: 7


Running A* tests on puzzle:
[[1 1 0]
 [1 0 0]
 [1 1 1]]
A* (manhattan_heuristic): 0.030 seconds
solution: [(2, 0), (1, 2), (1, 1), (2, 0), (1, 0), (0, 1), (0, 0)]
steps: 7
nodes_checked: 206


A* (lights_on_heuristic): 0.052 seconds
solution: [(0, 1), (1, 2)

In [425]:
from tabulate import tabulate

table_data = []

for j in range(6):
    for result in results:
        for i in range(6):  
            table_data.append(
                [
                "\n".join(" ".join(str(cell) for cell in row) for row in result[0]),    
                #tests[j],
                f"Name: {result[i+1]['name']}\n"  # Heuristic name
                f"Nodes Checked: {result[i+1]['nodes_checked']}\n"  # Nodes checked
                f"Steps: {result[i+1]['steps']}\n"  # Number of steps
                f"Time: {result[i+1]['time']:.3f} sec \n"
                f"Solution: {result[i+1]['solution']}"    
                ]
        )

# Display the table
print("Results:")
print(
    tabulate(
        table_data,
        headers=[
            "Test",
            "Algorithm"
            #*[f"A* ({heuristic.__name__})" for heuristic in heuristics],
            #*[f"Weighted A* ({heuristic.__name__}, alpha = {weight})" for heuristic in weighted_heuristics for weight in weights]
        ],
        floatfmt=".3f",
        numalign="center",
        stralign="center",
        tablefmt="fancy_grid" 
    )
)


Results:
╒═════════╤════════════════════════════════════════════════════════════════════════════════════════════════════╕
│  Test   │                                             Algorithm                                              │
╞═════════╪════════════════════════════════════════════════════════════════════════════════════════════════════╡
│  1 1 1  │                                   Name: A* (manhattan_heuristic)                                   │
│  1 1 1  │                                          Nodes Checked: 3                                          │
│  1 0 0  │                                              Steps: 2                                              │
│         │                                          Time: 0.001 sec                                           │
│         │                                     Solution: [(0, 2), (1, 0)]                                     │
├─────────┼────────────────────────────────────────────────────────────────────────────

In [426]:
result[8]

IndexError: tuple index out of range